In [ ]:
"""
Created on Tue Jan 17 10:02 2022

Look at the Smith T and S profiles

Author: @claraburgard

"""

In [ ]:
import numpy as np
import xarray as xr
from tqdm.notebook import trange, tqdm
import matplotlib.pyplot as plt
import basal_melt_param.useful_functions as uf

In [ ]:
%matplotlib qt5

In [ ]:
nemo_run = 'bi646' # 'bf663','bi646' 

In [ ]:
inputpath_data='/bettik/burgardc/DATA/NN_PARAM/interim/SMITH_'+nemo_run+'/'
inputpath_mask='/bettik/burgardc/DATA/NN_PARAM/interim/ANTARCTICA_IS_MASKS/SMITH_'+nemo_run+'/'
inputpath_profiles='/bettik/burgardc/DATA/NN_PARAM/interim/T_S_PROF/SMITH_'+nemo_run+'/'
inputpath_plumes='/bettik/burgardc/DATA/NN_PARAM/interim/PLUMES/SMITH_'+nemo_run+'/'
inputpath_boxes='/bettik/burgardc/DATA/NN_PARAM/interim/BOXES/SMITH_'+nemo_run+'/'


outputpath_melt = '/bettik/burgardc/DATA/NN_PARAM/interim/SMITH_'+nemo_run+'/'
outputpath_nn = '/bettik/burgardc/DATA/NN_PARAM/interim/INPUT_DATA/SMITH_'+nemo_run+'_EXTRAPDRAFT_CHUNKS/'
inputpath_tides = '/bettik/burgardc/DATA/BASAL_MELT_PARAM/interim/TIDES/'

In [ ]:
file_allTS_Ant = xr.open_dataset(inputpath_data + '3D_variables_of_interest_allyy_Ant.nc')
file_allTS_Ant_stereo = xr.open_dataset(inputpath_data + '3D_variables_of_interest_allyy_Ant_stereo.nc')

In [ ]:
file_allTS_Ant['so'].isel(time=32).isel(deptht=30).plot()

In [ ]:
map_lim = [-3000000,3000000]

file_allTS = xr.open_dataset(inputpath_data + '3D_variables_of_interest_allyy_Ant_stereo.nc')
file_TS_oney = xr.open_dataset(inputpath_profiles + 'T_S_theta_ocean_corrected_yy32.nc')
file_bathy_allyy = xr.open_mfdataset(inputpath_data+'corrected_draft_bathy_isf.nc')
file_bathy_allyy = uf.cut_domain_stereo(file_bathy_allyy, map_lim, map_lim)


In [ ]:
mask_test = xr.open_dataset(inputpath_data + 'mask_for_ocean_through_salinity.nc')

In [ ]:
mask_test['so'].isel(time=32).isel(deptht=30).plot()

In [ ]:
file_allTS_Ant

In [ ]:
file_bathy_32 = file_bathy_allyy.isel(time=32)

In [ ]:
plt.figure()
file_bathy_32['corrected_isf_bathy'].where(file_bathy_32['corrected_isf_bathy'] > 564.862).plot()

In [ ]:
sal_dep40 = file_TS_oney['salinity_ocean'].isel(depth=40)


In [ ]:
plt.figure()
sal_dep40.where(sal_dep40 > 0).plot()

In [ ]:
sal_test = file_allTS['so'].isel(time=range(40))

In [ ]:
plt.figure()
sal_test.isel(time=32,deptht=40).plot(vmin=33)

In [ ]:
tt = 0
file_isf_orig = xr.open_dataset(inputpath_mask+'nemo_5km_isf_masks_and_info_and_distance_oneFRIS_yy'+str(tt).zfill(2)+'.nc')
nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
large_isf = file_isf_nonnan['Nisf'].where(file_isf_nonnan['isf_area_here'] >= 2500, drop=True)
file_isf = file_isf_nonnan.sel(Nisf=large_isf)


In [ ]:
file_isf_list = []
for tt in range(8):
    file_isf_list.append(xr.open_dataset(inputpath_mask+'nemo_5km_isf_masks_and_info_and_distance_oneFRIS_yy'+str(tt).zfill(2)+'.nc'))
file_isf_allyy = xr.concat(file_isf_list, dim='time')   

In [ ]:
file_TS_list = []
for tt in range(1970,1978):
    file_TS_list.append(xr.open_dataset(inputpath_profiles+'T_S_mean_prof_corrected_km_contshelf_'+str(tt).zfill(2)+'.nc'))
file_TS_allyy = xr.concat(file_TS_list, dim='time')                     


In [ ]:
file_TS_allyy

In [ ]:
for kisf in file_TS_allyy.Nisf:
    plt.figure()
    file_TS_allyy['theta_ocean']

In [ ]:
dom = 50

f = plt.figure()
f.set_size_inches(8.25*1.5, 8.25*1.5)

ax={}

leg_hdl = []

i = 0
    
for kisf in tqdm(file_TS_allyy.Nisf):
    
    ax[i] = f.add_subplot(8,5,i+1)
    kisf_data = file_TS_allyy['salinity_ocean'].sel(Nisf=kisf)

    for tt in file_TS_allyy.time:
    
        kisf_tt_data = kisf_data.sel(time=tt).values

        leg, = plt.plot(kisf_tt_data,-1*file_TS_allyy.depth, color='grey', alpha=0.1)

    ax[i].set_title(str(file_isf_allyy['isf_name'].sel(time=tt-1970,Nisf=kisf).values))
    #ax[i].set_xlim(-2.2,2.5)
    ax[i].set_xlim(32,35)

    i = i+1
        

f.subplots_adjust(bottom=0.05, wspace=0.1)

#f.legend(handles = leg_hdl , labels=['REALISTIC', 'COLDAMU', 'WARMROSS', 'WARMGETZ', 'NEW_REALISTIC', '2200', '2300'], loc='lower center', ncol=4)
#f.legend(handles = leg_hdl , labels=['REALISTIC', 'COLDAMU', 'WARMROSS', 'HIGHGETZ'], loc='lower center', ncol=4)#, 'NEW_REALISTIC', '2200', '2300'], loc='lower center', ncol=4)

f.tight_layout()
sns.despine()


In [ ]:
file_TS_allyy['salinity_ocean'].sel(Nisf=66)

In [ ]:
file_isf['isf_name']

In [ ]:
isf_dim = [10,11,12,13,18,22,23,24,25,30,31,33,38,39,40,42,43,44,45,47,48,51,52,53,54,55,58,61,65,66,69,70,71,73,75]
isf_dim_new = file_isf.Nisf.values

In [ ]:
for kisf in range(76):
    if (kisf in isf_dim) and (kisf not in isf_dim_new):
        plt.figure()
        file_isf_orig['ISF_mask'].where(file_isf_orig['ISF_mask'] == kisf, drop=True).plot()
        plt.title(file_isf_orig['isf_name'].sel(Nisf=kisf).values)

    

In [ ]:
file_isf_list = []
for tt in range(40):
    file_isf_list.append(xr.open_dataset(inputpath_mask+'nemo_5km_isf_masks_and_info_and_distance_oneFRIS_yy'+str(tt).zfill(2)+'.nc'))
file_isf_allyy = xr.concat(file_isf_list, dim='time')                     


In [ ]:
kisf = 66
for tt in range(40):
    tt_GL = file_isf_allyy['GL_mask'].sel(time=tt)
    tt_GL.where(tt_GL == kisf).plot(add_colorbar=False)

In [ ]:
NEMO_melt_rates_2D = xr.open_dataset(outputpath_melt+'melt_rates_2D_NEMO_yy'+str(tt).zfill(2)+'.nc')


In [ ]:
NEMO_melt_rates_2D['melt_m_ice_per_y'].plot(vmax=30)

In [ ]:
file_isf_orig['ISF_mask'].plot()

In [ ]:
plt.close('all')

In [ ]:
(file_isf_orig['ISF_mask'] == kisf).plot()
